In [47]:
import pandas as pd
import collections

from etl_gestores import etl as etlgest
from etl_gestores import etl_nota
from etl_idesp import etl as etlidesp

ImportError: cannot import name 'etl_nota' from 'etl_gestores' (D:\DESAFIO_SEDUC\etl_gestores.py)

In [2]:
#https://dados.educacao.sp.gov.br/search/field_topic/desafios-seduc-164

In [3]:
df_gestao = etl_nota()

In [4]:
df_gestao.head(3)

,id_gestao,id_escola,id_gestor,ano_gestao,diretoria,funcao,nota_var,n_atuacoes
0,esc-902724-ano-2017,902724,288268,2017,LESTE 3,DIRETOR,5.5,2
1,esc-902724-ano-2018,902724,288268,2018,LESTE 3,DIRETOR,5.5,2
2,esc-13006-ano-2015,13006,385732,2015,GUARATINGUETA,DIRETOR,10.0,4


In [3]:
df_gestores = etlgest()

In [4]:
df_idesp = etlidesp()

In [5]:
result = pd.merge(df_idesp,
                  df_gestores,
                  on='id_gestao')

In [6]:
nota_gestor = {}
nota_gestor['id_gestor'] = []
nota_gestor['nota_var'] = []
nota_gestor['n_atuacoes'] = []
for nome, dados in result.groupby('id_gestor'):
    nota_gestor['id_gestor'].append(nome)
    nota_gestor['nota_var'].append(dados.variacao.mean())
    nota_gestor['n_atuacoes'].append(len(dados))

nota_gestor = pd.DataFrame(data=nota_gestor)

In [7]:
nota_gestor.head(3)

,id_gestor,nota_var,n_atuacoes
0,131346,0.636364,11
1,131376,-0.428571,7
2,131393,8.333333,6


In [8]:
result2 = pd.merge(nota_gestor,
                   df_gestores,
                   on='id_gestor')

In [9]:
nota_gestor[nota_gestor.id_gestor == 'NAO']

C:\Users\admprime\Miniconda3\envs\fklearn\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,id_gestor,nota_var,n_atuacoes


In [10]:
result2[result2.id_gestor == 403084]

,id_gestor,nota_var,n_atuacoes,id_gestao,id_escola,ano_gestao,diretoria,funcao
167412,403084,4.4,5,esc-906554-ano-2014,906554,2014,SUL 2,COORDENADOR
167413,403084,4.4,5,esc-906554-ano-2015,906554,2015,SUL 2,COORDENADOR
167414,403084,4.4,5,esc-906554-ano-2016,906554,2016,SUL 2,COORDENADOR
167415,403084,4.4,5,esc-906554-ano-2017,906554,2017,SUL 2,COORDENADOR
167416,403084,4.4,5,esc-906554-ano-2018,906554,2018,SUL 2,COORDENADOR


# FATO
- Avaliacao do Saresp
- Ano
- Valor (medprof)
- Dim disciplina (serie/periodo/conteudo) - TODO: incluir o professor aqui.
- Dim equipe de gestores - tabela com Ids e uma auxiliar de detalhamentos
- Dim escola

# FUNCOES

# CARREGAMENTO DOS DADOS

In [12]:
# GESTORES
arquivo = 'dados\lideres\lideres_DIRETORES DE ESCOLA.csv'
diretor = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_VICE_DIRETOR.csv'
vice = pd.read_csv(arquivo, sep=';')
arquivo = 'dados\lideres\lideres_PROFESSOR_COORDENADOR.csv'
coordenador = pd.read_csv(arquivo, sep=';')
vice = vice.drop('Unnamed: 16',axis=1)

In [19]:
df_gestores = etl()

In [20]:
df_gestores

,id_gestao,id_escola,id_gestor,ano_gestao,diretoria,funcao
0,esc-902724-ano-2017,902724,288268,2017,LESTE 3,DIRETOR
1,esc-902724-ano-2018,902724,288268,2018,LESTE 3,DIRETOR
2,esc-13006-ano-2015,13006,385732,2015,GUARATINGUETA,DIRETOR
3,esc-13006-ano-2016,13006,385732,2016,GUARATINGUETA,DIRETOR
4,esc-13006-ano-2017,13006,385732,2017,GUARATINGUETA,DIRETOR
5,esc-13006-ano-2018,13006,385732,2018,GUARATINGUETA,DIRETOR
6,esc-42250-ano-2014,42250,385733,2014,PINDAMONHANGABA,DIRETOR
7,esc-42250-ano-2015,42250,385733,2015,PINDAMONHANGABA,DIRETOR
8,esc-10194-ano-2014,10194,385734,2014,TABOAO DA SERRA,DIRETOR
9,esc-10194-ano-2015,10194,385734,2015,TABOAO DA SERRA,DIRETOR


### ESTUDO DAS VARIACOES

In [320]:
idesp_group[idesp_group.variacao > 200]

,id_gestao,id_escola,ano_gestao,nota_ano,variacao
1033,esc-12488-ano-2015,12488,2015,2.87,304
4102,esc-2008-ano-2016,2008,2016,2.46,261
7679,esc-29269-ano-2015,29269,2015,7.96,237
11831,esc-40551-ano-2016,40551,2016,2.25,324
12296,esc-4157-ano-2014,4157,2014,3.50,379
12667,esc-42500-ano-2018,42500,2018,3.70,345
12717,esc-42626-ano-2018,42626,2018,3.08,201


In [321]:
df_idesp[df_idesp.loc[:,'CODIGO CIE'] == 40551]

,CODIGO CIE,CODIGO INEP,DIRETORIA,ESCOLA,MUNICIPIO,NIVEL ENSINO,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
2253,40551,35040551.0,ITAPEVI,LENIO VIEIRA DE MORAES PROFESSOR ...,BARUERI,ENSINO MEDIO,0.76,1.14,0.96,0.86,1.3,1.15,1.27,1.0,0.53,2.25,1.77,1.56


# GERANDO DIMENSAO GESTORES
- cd_escola
- ano
- diretoria
- id_diretor - detalhamento aparece em outro lugar
- id_vice
- id_coordenador

In [8]:
LISTA_ANOS = ['2014', '2015', '2016', '2017', '2018']
def foi_gestor(ano):
    return ano == 'SIM'

In [13]:
def limpeza(tabela_extraida, funcao_gestor, df_gestao):
    for _, row in tabela_extraida.iterrows():
        for ANO in LISTA_ANOS:
            if foi_gestor(row.loc[ANO]):
                df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
                df_gestao['id_escola'].append(row.CD_ESCOLA)
                df_gestao['id_gestor'].append(row.id_interno)
                df_gestao['ano_gestao'].append(ANO)
                df_gestao['diretoria'].append(row.DIRETORIA)
                df_gestao['funcao'].append(funcao_gestor)
    return df_gestao

In [14]:
df_gestao = {}
df_gestao['id_gestao'] = []
df_gestao['id_escola'] = []
df_gestao['id_gestor'] = []
df_gestao['ano_gestao'] = []
df_gestao['diretoria'] = []
df_gestao['funcao'] = []

df_gestao = limpeza(diretor, 'DIRETOR', df_gestao)
df_gestao = limpeza(vice, 'VICE', df_gestao)
df_gestao = limpeza(coordenador, 'COORDENADOR', df_gestao)

In [15]:
df_gestao = pd.DataFrame(df_gestao)

In [ ]:
df_gestao = {}
df_gestao['id_gestao'] = []
df_gestao['id_escola'] = []
df_gestao['id_gestor'] = []
df_gestao['ano_gestao'] = []
df_gestao['diretoria'] = []
df_gestao['funcao'] = []

for _, row in diretor.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('DIRETOR')

for _, row in vice.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('VICE')
            
for _, row in coordenador.iterrows():
    for ANO in LISTA_ANOS:
        if foi_gestor(row.loc[ANO]):
            df_gestao['id_gestao'].append('esc-' + str(row.CD_ESCOLA) + '-ano-' + str(ANO))
            df_gestao['id_escola'].append(row.CD_ESCOLA)
            df_gestao['id_gestor'].append(row.id_interno)
            df_gestao['ano_gestao'].append(ANO)
            df_gestao['diretoria'].append(row.DIRETORIA)
            df_gestao['funcao'].append('COORDENADOR')
            
df_gestao = pd.DataFrame(df_gestao)

### VERIFICANDO QUANTIDADE DE TROCAS DE GESTORES

In [ ]:
k = 0
contagem_gestores = collections.Counter(df_gestao.id_gestao)
for id_gestao in contagem_gestores:
    if contagem_gestores[id_gestao] == 3:
        k += 1

In [323]:
df_gestao[df_gestao.id_gestao == 'esc-472505-ano-2014']

,id_gestao,id_escola,id_gestor,ano_gestao,diretoria,funcao
28754,esc-472505-ano-2014,472505,334929,2014,SUL 3,DIRETOR
95938,esc-472505-ano-2014,472505,222988,2014,SUL 3,VICE
140523,esc-472505-ano-2014,472505,399478,2014,SUL 3,COORDENADOR


# GESTAO + IDESP

In [390]:
cont_gestores = {}
cont_gestores['id_gestao'] = []
cont_gestores['id_escola'] = []
cont_gestores['n_gestores'] = []

for nome, dados in df_gestao.groupby('id_gestao'):
    n_gestores = len(dados)
    cont_gestores['id_gestao'].append(nome)
    cont_gestores['id_escola'].append(dados.id_escola.iloc[0])
    cont_gestores['n_gestores'].append(n_gestores)
    
cont_gestores = pd.DataFrame(data=cont_gestores)
cont_gestores_aux = {}
cont_gestores_aux['id_escola'] = []
cont_gestores_aux['mediana'] = []
for nome, dados in cont_gestores.groupby('id_escola'):
    cont_gestores_aux['id_escola'].append(dados.id_escola.iloc[0])
    cont_gestores_aux['mediana'].append(dados.n_gestores.median())

cont_gestores_aux = pd.DataFrame(data=cont_gestores_aux)
cont_gestores = pd.merge(cont_gestores,
                  cont_gestores_aux,
                  on='id_escola')
cont_gestores['var_gestores'] = result.n_gestores - result.mediana

result = pd.merge(idesp_group,
                  cont_gestores,
                  on='id_gestao')
result.loc[:,['variacao','var_gestores']].corr()

# DIFERENTE DO QUE EU PENSAVA, PARECE QUE ESSE NUMERO ALTO DE GESTORES
# MAIS OU MENOS NORMAL E NAO IMPACTA MUITO OS RESULTADOS

,variacao,var_gestores
variacao,1.000000,0.006077
var_gestores,0.006077,1.000000


# NOTAS DOS GESTORES

In [492]:
result = pd.merge(idesp_group,
                  df_gestao,
                  on='id_gestao')

In [493]:
result.head(5)

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
0,esc-10005-ano-2014,10005,2014,3.48,10,10005,326625,2014,ITAPECERICA DA SERRA,DIRETOR
1,esc-10005-ano-2014,10005,2014,3.48,10,10005,157667,2014,ITAPECERICA DA SERRA,VICE
2,esc-10005-ano-2014,10005,2014,3.48,10,10005,314895,2014,ITAPECERICA DA SERRA,VICE
3,esc-10005-ano-2014,10005,2014,3.48,10,10005,351780,2014,ITAPECERICA DA SERRA,COORDENADOR
4,esc-10005-ano-2014,10005,2014,3.48,10,10005,398436,2014,ITAPECERICA DA SERRA,COORDENADOR


In [494]:
nota_gestor = {}
nota_gestor['id_gestor'] = []
nota_gestor['nota_var'] = []
nota_gestor['n_atuacoes'] = []
for nome, dados in result.groupby('id_gestor'):
    nota_gestor['id_gestor'].append(nome)
    nota_gestor['nota_var'].append(dados.variacao.mean())
    nota_gestor['n_atuacoes'].append(len(dados))

nota_gestor = pd.DataFrame(data=nota_gestor)

In [495]:
nota_gestor.nota_var.describe()

count    50969.000000
mean         6.537152
std          8.153998
min        -13.000000
25%          2.000000
50%          6.166667
75%         10.500000
max         30.000000
Name: nota_var, dtype: float64

In [496]:
nota_gestor.n_atuacoes.describe()

count    50969.000000
mean         3.251310
std          1.807438
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max         22.000000
Name: n_atuacoes, dtype: float64

In [514]:
nota_gestor = nota_gestor[nota_gestor.n_atuacoes >= 2]

In [516]:
nota_gestor.head(5)

,id_gestor,nota_var,n_atuacoes
0,131346,-1.250000,4
1,131376,6.500000,2
2,131393,8.333333,6
3,131418,7.400000,5
4,131444,12.428571,7


In [498]:
result[result.id_gestor == 355299]

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
65554,esc-3554-ano-2015,3554,2015,3.38,0,3554,355299,2015,CENTRO,DIRETOR
65563,esc-3554-ano-2016,3554,2016,3.92,15,3554,355299,2016,CENTRO,DIRETOR
65574,esc-3554-ano-2017,3554,2017,2.75,-13,3554,355299,2017,CENTRO,DIRETOR
65584,esc-3554-ano-2018,3554,2018,3.89,30,3554,355299,2018,CENTRO,DIRETOR


In [503]:
dados

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
157468,esc-923771-ano-2016,923771,2016,2.350,-13,923771,SIM,2016,GUARULHOS NORTE,VICE
157479,esc-923771-ano-2017,923771,2017,2.945,25,923771,SIM,2017,GUARULHOS NORTE,VICE
157487,esc-923771-ano-2018,923771,2018,2.645,-10,923771,SIM,2018,GUARULHOS NORTE,VICE


# DIMENSAO SARESP
- Substituida pela dimensao IDESP

In [ ]:
def limpa_serie_ano(serie_ano):
    if serie_ano == '9Âº Ano EF' or serie_ano == '9º Ano EF':
        return 'EF-9'
    elif serie_ano == '7Âº Ano EF':
        return 'EF-7'
    elif serie_ano == 'EM-3Âª sÃ©rie' or serie_ano == 'EM-3ª série':
        return 'EM-3'
    elif serie_ano == '3Âº Ano EF' or serie_ano == '3º Ano EF':
        return 'EF-3'
    elif serie_ano == '5Âº Ano EF' or serie_ano == '5º Ano EF':
        return 'EF-5'
    
def limpa_ds_comp(ds_comp):
    if ds_comp == 'LÃ\x8dNGUA PORTUGUESA' or ds_comp == 'LÍNGUA PORTUGUESA':
        return 'PORTUGUES'
    elif ds_comp == 'MATEMÃ\x81TICA' or ds_comp == 'MATEMÁTICA':
        return 'MATEMATICA'
    elif ds_comp == 'CIÃ\x8aNCIAS':
        return 'CIENCIAS'    
    
def enulo(valor):
    return str(valor) == 'nan'   

In [ ]:
# SARESP
arquivo = 'dados/saresp/SARESP_escolas_2014.csv'
df_2014 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2014['ano'] = 2014
arquivo = 'dados/saresp/SARESP_escolas_2015.csv'
df_2015 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2015['ano'] = 2015
arquivo = 'dados/saresp/SARESP_escolas_2016.csv'
df_2016 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2016['ano'] = 2016
arquivo = 'dados/saresp/SARESP_escolas_2017.csv'
df_2017 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2017['ano'] = 2017
arquivo = 'dados/saresp/SARESP_escolas_2018.csv'
df_2018 = pd.read_csv(arquivo, sep=';', encoding='latin-1')
df_2018['ano'] = 2018
saresp = df_2014.append(df_2015).append(df_2016).append(df_2017).append(df_2018)
saresp = saresp[saresp.medprof.notnull()] # nao estou interessado se nao tiver o valor do saresp

In [ ]:
saresp.SERIE_ANO = saresp.SERIE_ANO.apply(limpa_serie_ano)
saresp.ds_comp = saresp.ds_comp.apply(limpa_ds_comp)
saresp.medprof = saresp.medprof \
                .apply(lambda x: str(x).replace('.','').replace(',','.')) \
                .astype(float) 

df_saresp = {}
df_saresp['id_gestao'] = []
df_saresp['id_escola'] = []
df_saresp['ano_gestao'] = []
df_saresp['serie_ano'] = []
df_saresp['periodo'] = []
df_saresp['disciplina'] = []
df_saresp['medprofici'] = []

for _, row in saresp.iterrows():
    df_saresp['id_gestao'].append('esc-' + str(row.CODESC) + '-ano-' + str(row.ano))
    df_saresp['id_escola'].append(row.CODESC)
    df_saresp['ano_gestao'].append(row.ano)
    df_saresp['serie_ano'].append(row.SERIE_ANO)
    df_saresp['periodo'].append(row.periodo)
    df_saresp['disciplina'].append(row.ds_comp)
    df_saresp['medprofici'].append(row.medprof)

df_saresp = pd.DataFrame(data=df_saresp) 

saresp_medias = {}
saresp_medias['id_gestao'] = []
saresp_medias['media'] = []
saresp_medias['id_escola'] = []
for nome, dados in df_saresp.groupby('id_gestao'):
    saresp_medias['id_gestao'].append(nome)
    saresp_medias['media'].append(dados.medprofici.mean())
    saresp_medias['id_escola'].append(dados.id_escola.iloc[0])

saresp_medias = pd.DataFrame(data = saresp_medias)

In [48]:
import pandas as pd
import collections

import etl_gestores
import etl_idesp

In [49]:
dados = etl_gestores.etl()

In [50]:
dados.head(3)

,id_gestao,id_escola,id_gestor,ano_gestao,diretoria,funcao
0,esc-902724-ano-2017,902724,288268,2017,LESTE 3,DIRETOR
1,esc-902724-ano-2018,902724,288268,2018,LESTE 3,DIRETOR
2,esc-13006-ano-2015,13006,385732,2015,GUARATINGUETA,DIRETOR


In [51]:
df_idesp = etl_idesp.etl()

In [52]:
mergeidesp = pd.merge(df_idesp, dados, on='id_gestao')

In [53]:
mergeidesp.head(3)

,id_gestao,id_escola_x,ano_gestao_x,nota_ano,variacao,anterior_nulo,atual_nulo,unidades_diretoria,nome_diretoria,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,326625,2014,ITAPECERICA DA SERRA,DIRETOR
1,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,157667,2014,ITAPECERICA DA SERRA,VICE
2,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,314895,2014,ITAPECERICA DA SERRA,VICE


In [54]:
mergeidesp = mergeidesp.dropna(subset=['nota_ano'])
mergeidesp = mergeidesp.rename({'ano_gestao_x': 'ano_gestao'}, axis=1)
mergeidesp.ano_gestao = mergeidesp.ano_gestao.astype(int)

In [55]:
mergeidesp[mergeidesp.id_gestor == 326625]

,id_gestao,id_escola_x,ano_gestao,nota_ano,variacao,anterior_nulo,atual_nulo,unidades_diretoria,nome_diretoria,id_escola_y,id_gestor,ano_gestao_y,diretoria,funcao
0,esc-10005-ano-2014,10005,2014,3.48,10.828025,0,1,133,ITAPECERICA DA SERRA,10005,326625,2014,ITAPECERICA DA SERRA,DIRETOR
5,esc-10005-ano-2015,10005,2015,3.78,8.620690,0,1,133,ITAPECERICA DA SERRA,10005,326625,2015,ITAPECERICA DA SERRA,DIRETOR
11,esc-10005-ano-2016,10005,2016,3.26,-13.756614,0,1,133,ITAPECERICA DA SERRA,10005,326625,2016,ITAPECERICA DA SERRA,DIRETOR
17,esc-10005-ano-2017,10005,2017,3.80,16.564417,0,1,133,ITAPECERICA DA SERRA,10005,326625,2017,ITAPECERICA DA SERRA,DIRETOR
21,esc-10005-ano-2018,10005,2018,3.83,0.789474,0,1,133,ITAPECERICA DA SERRA,10005,326625,2018,ITAPECERICA DA SERRA,DIRETOR


In [56]:
LISTA_ANOS = [2014, 2015, 2016, 2017, 2018]
adic_media = {}
adic_media['id_gestao'] = []
adic_media['media_variacao'] = []
adic_media['media_gestor'] = []

for nome,dados in mergeidesp.groupby('id_gestor'):
    nota_total = 0
    variacao_total = 0
    contagem = 0
    lista_anos = dados.ano_gestao.tolist()
    lista_anos.sort()
    primeira_gestao = lista_anos[0]
    nota_ano_antes = 0
    variacao_ano_antes = 0
    for ano in lista_anos:
        if ano == primeira_gestao:
            row = dados[dados.ano_gestao == primeira_gestao].iloc[0]
            nota_ano_antes = row.nota_ano
            variacao_ano_antes = row.variacao
            continue
            #nota_anterior = row.nota_ano * (1 - row.variacao/100)
            #variacao_anterior = 0
        else:
            nota_anterior = nota_ano_antes
            variacao_anterior = variacao_ano_antes
            row = dados[dados.ano_gestao == ano].iloc[0]
            nota_ano_antes = row.nota_ano
            variacao_ano_antes = row.variacao
            
        nota_total += nota_anterior
        variacao_total += variacao_anterior
        contagem += 1

        row = dados[dados.ano_gestao == ano].iloc[0]
        adic_media['id_gestao'].append(row.id_gestao)
        adic_media['media_gestor'].append(nota_total / contagem)
        adic_media['media_variacao'].append(variacao_total / contagem)
        
adic_media = pd.DataFrame(data=adic_media)

In [57]:
mergeidesp2 = pd.merge(mergeidesp, adic_media, on='id_gestao', how='left')

In [58]:
mergeidesp2 = mergeidesp2.drop(mergeidesp2[mergeidesp2.ano_gestao_y == '2014'].index)

In [59]:
mergeidesp2.loc[:,['nota_ano','media_variacao','media_gestor']].corr()

,nota_ano,media_variacao,media_gestor
nota_ano,1.000000,0.024101,0.822983
media_variacao,0.024101,1.000000,0.083194
media_gestor,0.822983,0.083194,1.000000
